In [ ]:
from google.colab import auth
auth.authenticate_user()

project_id = "polar-ensign-432610-t7"
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
!pip install google-cloud-bigquery pandas scikit-learn Flask joblib flask-cors

In [ ]:
%%writefile app.py

from flask import Flask, request, jsonify
from flask_cors import CORS
from google.cloud import bigquery
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.impute import SimpleImputer

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# Set up BigQuery client
client = bigquery.Client()

def fetch_player_data():
    query = """
    SELECT
        Player, Pos AS Position,
        `Per 90 Minutes - Gls` AS `Goals per 90`,
        `Per 90 Minutes - Ast` AS `Assists per 90`,
        `Standard - Sh_90` AS `Shots per 90`,
        `Standard - SoT_90` AS `Shots on Target per 90`,
        `Total - Cmp%` AS `Pass Completion %`,
        `Take-Ons - Succ%` AS `Dribbles Completed per 90`,
        `Tackles - TklW` AS `Tackles Won per 90`,
        `Int` AS `Interceptions per 90`,
        `Challenges - Tkl%` AS `Aerial Duels Won per 90`
    FROM `polar-ensign-432610-t7.squadsight_scouting.2023-2024`
    WHERE Pos IS NOT NULL
    """

    query_job = client.query(query)
    df = query_job.to_dataframe()

    # Log the DataFrame to inspect if data is fetched correctly
    if df is None or df.empty:
        print("No data fetched from BigQuery.")
    else:
        print(f"Fetched {len(df)} rows from BigQuery.")

    return df


# Preprocess data
def preprocess_data(df):
    rank_columns = [
        'Goals per 90', 'Assists per 90', 'Shots per 90', 'Shots on Target per 90',
        'Pass Completion %', 'Dribbles Completed per 90', 'Tackles Won per 90',
        'Interceptions per 90', 'Aerial Duels Won per 90'
    ]

    # Impute missing values
    imputer = SimpleImputer(strategy='mean')
    df[rank_columns] = imputer.fit_transform(df[rank_columns])

    # Scale the data
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df[rank_columns])

    # Apply K-means clustering
    kmeans = KMeans(n_clusters=10, random_state=42)
    df['Cluster'] = kmeans.fit_predict(scaled_data)

    return df, rank_columns

# Fetch player_id from BigQuery based on player name
def fetch_player_ids(player_names):
    placeholders = ', '.join(['@name' + str(i) for i in range(len(player_names))])
    query = f"""
    SELECT player_id, name
    FROM `polar-ensign-432610-t7.transfermarkt.players`
    WHERE name IN ({placeholders})
    """
    query_params = {f'name{i}': name for i, name in enumerate(player_names)}

    query_job = client.query(query, job_config=bigquery.QueryJobConfig(
        query_parameters=[bigquery.ScalarQueryParameter(name, "STRING", value)
                          for name, value in query_params.items()]
    ))

    results = query_job.to_dataframe()
    return results

# Endpoint to find similar players
@app.route('/similar-players', methods=['GET'])
def find_similar_players():
    player_name = request.args.get('player_name')
    num_similar = int(request.args.get('num_similar', 5))

    # Fetch and preprocess data
    df = fetch_player_data()
    df, rank_columns = preprocess_data(df)

    if player_name not in df['Player'].values:
        return jsonify({"error": "Player not found"}), 404

    # Find similar players
    player_data = df[df['Player'] == player_name]
    position = player_data['Position'].values[0]
    cluster = player_data['Cluster'].values[0]

    same_position_cluster = df[(df['Position'] == position) & (df['Cluster'] == cluster)]
    distances = pairwise_distances(player_data[rank_columns], same_position_cluster[rank_columns])

    same_position_cluster['Distance'] = distances.flatten()
    similar_players = same_position_cluster.nsmallest(num_similar + 1, 'Distance')
    similar_players = similar_players[similar_players['Player'] != player_name]

    # Fetch player_ids for the similar players
    similar_player_names = similar_players['Player'].tolist()
    player_ids_df = fetch_player_ids(similar_player_names)

    # Merge player names with their IDs
    merged_data = pd.merge(similar_players, player_ids_df, left_on='Player', right_on='name', how='left')
    result = merged_data[['Player', 'player_id']].to_dict(orient='records')

    # Return the results
    return jsonify(result)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)


Writing app.py


In [ ]:
%%writefile requirements.txt
Flask
google-cloud-bigquery
pandas
scikit-learn
db-dtypes
flask-cors

Writing requirements.txt


In [ ]:
%%writefile Dockerfile
# Use an official Python runtime as a parent image
FROM python:3.8-slim

# Set the working directory in the container
WORKDIR /app

# Copy the current directory contents into the container at /app
COPY . /app

# Install any needed packages specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Make port 8080 available to the world outside this container
EXPOSE 8080

# Run app.py when the container launches
CMD ["python", "app.py"]

Overwriting Dockerfile


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!gcloud auth login
!gcloud config set project polar-ensign-432610-t7


You are running on a Google Compute Engine virtual machine.
It is recommended that you use service accounts for authentication.

You can run:

  $ gcloud config set account `ACCOUNT`

to switch accounts if necessary.

Your credentials may be visible to others with access to this
virtual machine. Are you sure you want to authenticate with
your personal account?

Do you want to continue (Y/n)?  T
Please enter 'y' or 'n':  Y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googl

In [ ]:
!gcloud builds submit --tag gcr.io/polar-ensign-432610-t7/playerknn .

Creating temporary archive of 46 file(s) totalling 55.0 MiB before compression.
Uploading tarball of [.] to [gs://polar-ensign-432610-t7_cloudbuild/source/1724764668.83668-abea27abb94f4f08a7337e9daf5f1ba3.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/polar-ensign-432610-t7/locations/global/builds/715827e8-38a3-4053-90b4-c5cda90fc17a].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/715827e8-38a3-4053-90b4-c5cda90fc17a?project=116084333061 ].
Waiting for build to complete. Polling interval: 1 second(s).
 REMOTE BUILD OUTPUT
starting build "715827e8-38a3-4053-90b4-c5cda90fc17a"

FETCHSOURCE
Fetching storage object: gs://polar-ensign-432610-t7_cloudbuild/source/1724764668.83668-abea27abb94f4f08a7337e9daf5f1ba3.tgz#1724764678817882
Copying gs://polar-ensign-432610-t7_cloudbuild/source/1724764668.83668-abea27abb94f4f08a7337e9daf5f1ba3.tgz#1724764678817882...
/ [1 files][  6.5 MiB/  6.5 MiB]                                                
Operation c

In [ ]:
!gcloud run deploy --image gcr.io/polar-ensign-432610-t7/playerknn --platform managed --region eu-west2 --allow-unauthenticated

Service name (playerknn):  

Command killed by keyboard interrupt

^C


In [ ]:
from google.cloud import bigquery

# Explicitly set the project ID
client = bigquery.Client(project='polar-ensign-432610-t7')

query = """
    SELECT
        Player, Pos AS Position,
        `Per 90 Minutes - Gls` AS `Goals per 90`,
        `Per 90 Minutes - Ast` AS `Assists per 90`,
        `Standard - Sh_90` AS `Shots per 90`,
        `Standard - SoT_90` AS `Shots on Target per 90`,
        `Total - Cmp%` AS `Pass Completion %`,
        `Take-Ons - Succ%` AS `Dribbles Completed per 90`,
        `Tackles - TklW` AS `Tackles Won per 90`,
        `Int` AS `Interceptions per 90`,
        `Challenges - Tkl%` AS `Aerial Duels Won per 90`
    FROM `polar-ensign-432610-t7.squadsight_scouting.2023-2024`
    WHERE Pos IS NOT NULL
"""

query_job = client.query(query)
df = query_job.to_dataframe()

print(df.head())

               Player Position  Goals per 90  Assists per 90  Shots per 90  \
0  Tommaso Martinelli       GK           0.0             0.0           0.0   
1     Aaron Cresswell    DF,FW           0.0             0.0           0.0   
2      Aaron Ramsdale       GK           0.0             0.0           0.0   
3          Alex Matos       MF           0.0             0.0           0.0   
4         Alex Murphy    DF,FW           0.0             0.0           0.0   

   Shots on Target per 90  Pass Completion %  Dribbles Completed per 90  \
0                     0.0                NaN                        NaN   
1                     0.0               82.5                        0.0   
2                     0.0               69.7                        NaN   
3                     0.0                NaN                        NaN   
4                     0.0              100.0                        NaN   

   Tackles Won per 90  Interceptions per 90  Aerial Duels Won per 90  
0        

In [ ]:
pip install statsbombpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.2 MB/s eta 0:00:00
